In [3]:
import glob
import json
import pandas as pd
import ntpath
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
allfiles=glob.glob("C:/Users/Partha/Downloads/Task/test docs/*.tx?")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Partha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Partha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Partha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The following code finds out the list of most important most common words from a list of documents and writes it to a json file in the following format into a json which can be used from other web applications.
Table format 
Keyword | Docs | Sentences

Following steps summarize the tasks:
1) Read the text from the folder into doc _corpus
2) Perform the following steps to clean up the data
   a)Tokenize the sentences into words.
   b)Remove stopwords and irrelevant words like pronouns, as they wouldn't be good hash tag candidates.
   c) Use lemmatizer to group together different forms of the word to mean the same term like two and 2 should be considered the same in an NLP problem which filters the most important word.
   d) Use Stemmer to reduce reocurring verb forms to the root form like say,saying said gets reduced to say
   
3) Use Count Vectorizer to get a matrix with word  counts across the Corpus and find out relevant features.
4) Use TF IDF to find word relevance so that the frequency of the word in a document is compared to frquency of the word across the documents, this filters out irrelevant words by assigning them a low relevance score.
5) Combine the output features from count vectorizer and relevance matrix from TF IDF to generate the top 10 keywords
6) Create a Table to display the documents containing the keywords and filter out the sentences.
7) Convert output to a dictionary and write it to a JSON file.
   
   



In [ ]:
#importing the required packages
import nltk
import glob
import json
import pandas as pd
import ntpath
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


In [ ]:
#Initializing Variable and instantiating classes.
doc_corpus =pd.DataFrame()
filtered_data=pd.DataFrame()
name_list=[]
name_list_new=[]
doc_list=[]
sentence_list =[]
content = []
sent=[]
b=[]

stop_words = set( stopwords.words('english') + list(punctuation) + list(['good','let','how', 'mama','who','why','yo','\n'])+['reuter', '\x03'])
cv=CountVectorizer(stop_words=stop_words)

vectorizer = TfidfVectorizer(min_df=.3,max_features=10000,lowercase=True, analyzer='word',stop_words= 'english',ngram_range=(1,1))
lemmatizer = WordNetLemmatizer() 
porter = PorterStemmer()
tfid_trans=TfidfTransformer(smooth_idf=True,use_idf=True)
stop_words = set( stopwords.words('english') +  list(['good','let','who','why','how','\n'])+['reuter', '\x03'])

In [16]:
#Functions required for pre and post processing
def read_files() :
    #read all files from the folder and concatenate output into one list of texts
    allfiles=glob.glob("C:/Users/Partha/Downloads/Task/test docs/*.tx?")           
    for fname in allfiles:
     name_list.append(str(ntpath.basename(fname)))
     with open(fname,"r",encoding='utf-8-sig') as f:
      b=[str(f.read().splitlines())]
      sent.append(sent_tokenize(str(b)))
      content.append(b) 
    doc_corpus['name'] =name_list
    doc_corpus['content'] = content
    return (content)    

def tokenize(text):
    #tokenize the sentence into words and ignore unnecessary common words like pronouns 
    words=re.findall('[\w]+', text.lower())
    answer=[w for w in words if w not in stop_words and not w.isdigit()]
    return(answer)

def data_clean(text):
    #Adding code to clean data and remove words that are not ideal hashtag candidates
    token_text=[str(tokenize(str(content)))]
    lemma_txt=[str(lemmatizer.lemmatize(str(token_text)))]
    final_txt=[str(porter.stem(str(lemma_txt)))]
    return(final_txt) 

def sort_coo (coo_vector):
    # sortting the matrix containing the relevance of all words 
    tuples=zip(coo_vector.col,coo_vector.data)
    return(sorted(tuples,key=lambda x :(x[1],x[0]),reverse=True))
           
def extract_top10 (feat,sorted_coo):
    #filter out the top 10 most important matrices
    sorted_coo=sorted_coo[:10]
    score_v = []
    feat_value =[]
    for idx, score in sorted_coo:
        fname = feat[idx]
        #store feature name and its relevance score
        score_v.append(round(score, 3))
        feat_value.append(feat[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feat_value)):
        results[feat_value[idx]]=score_v[idx]
    
    return results
 
def search_keyword(keywords,doc_corpus):
 # Lookup the doc_corpus to find the documents and sentences containing the keyword   
 filtered_data=pd.DataFrame()
 for keyword in keywords.keys() :
   for idx,row in doc_corpus.iterrows():
    for line in str(row['content']).splitlines():
       words =set(tokenize(line))
       if keyword in words:
            name_list_new.append(row['name'])
            sentence_list.append(line)        
   mod_data=filtered_data.append({'keyword' :keyword, 'name' : str(name_list_new),'line':str(sentence_list)},ignore_index=True)
   filtered_data=mod_data
 return(mod_data)


In [ ]:
#Count Vectorizer and TF IDF at work
content=read_files()
final_txt=data_clean(content)
#Running count vectorizer on tokenized textto find the word counts
wordcount =cv.fit_transform(final_txt)
#The get_feature_names attribute gives the distinct important words in the text corpus
feat=cv.get_feature_names()
#Using a TF IDF Vectorizer to find the relation of the word frequency in relation to the frequenct of word across documents
tfid_trans.fit(wordcount)
tfidf_vec=tfid_trans.transform(cv.transform(final_txt))
#Following code is used to sort the matrix containing the relevance score of each word generated as a feature by Count Vectorizer
sorted_items=sort_coo(tfidf_vec.tocoo())
#Extracting top 10 keywords
keywords= extract_top10 (feat,sorted_items)
#Search the  documents and the sentences containing most important keywords
filtered_data=search_keyword(keywords,doc_corpus)
#Writing the collected data into a JSON file, which can be imported by other applications.
with open("file_output.txt",'w+') as json_file:
  json.dump(filtered_data.to_dict(),json_file)
